In [1]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import time

In [2]:
start = time.time()
data = pd.read_csv('/Users/sazid/Documents/vinlus dataset/GPS Spoofing Detection with Parallel Computing/aisdk-2006-03/aisdk_20060302.csv')

In [3]:
data1 = data.rename(columns = {'# Timestamp':'Timestamp'})
data1['Timestamp']=pd.to_datetime(data1['Timestamp'], format='%d/%m/%Y %H:%M:%S')
data1['ETA']= pd.to_datetime(data1['ETA'], format='%d/%m/%Y %H:%M:%S', errors='coerce')




In [4]:
data1 = data1[ ~(
    
    data1['ROT'].isnull() &
    data1['SOG'].isnull() &
    data1['Navigational status'].str.contains('Unknown value',case=False, na=False)
    
)] 

In [5]:
data1 = data1.drop(['IMO','Callsign','Cargo type','Ship type'], axis=1)

In [6]:
data1['SOG'].isnull().sum()

2532

In [7]:
data1['SOG'] = data1['SOG'].fillna(0)

In [8]:
data1 =  data1[((data1['Latitude']<=90) & (data1['Latitude']>= -90)) & ((data1['Longitude']<=180 ) & (data1['Longitude']>= -180))]

In [9]:
data1

,Timestamp,Type of mobile,MMSI,Latitude,Longitude,Navigational status,ROT,SOG,COG,Heading,Name,Width,Length,Type of position fixing device,Draught,Destination,ETA,Data source type
1,2006-03-02 00:00:00,Class A,257413900,63.105432,7.745262,Under way using engine,0.0,0.0,333.2,347.0,NaN,NaN,NaN,Undefined,NaN,NaN,NaT,AIS
2,2006-03-02 00:00:00,Class A,211219250,56.064118,7.616303,Under way using engine,12.9,14.4,15.0,16.0,NaN,NaN,NaN,Undefined,NaN,NaN,NaT,AIS
3,2006-03-02 00:00:00,Class A,257486000,56.909000,11.753333,Under way using engine,1.1,14.0,156.0,157.0,NaN,NaN,NaN,Undefined,NaN,NaN,NaT,AIS
7,2006-03-02 00:00:00,Class A,215034000,68.721683,13.343900,Under way using engine,NaN,8.2,228.4,NaN,NaN,NaN,NaN,Undefined,NaN,NaN,NaT,AIS
8,2006-03-02 00:00:00,Class A,232000190,54.384717,12.023633,Under way using engine,0.0,15.4,69.0,68.0,NaN,NaN,NaN,Undefined,NaN,NaN,NaT,AIS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711499,2006-03-02 23:59:59,Class A,311645000,59.678028,4.169532,Under way using engine,0.0,13.2,359.3,358.0,AXEL SPIRIT,44.0,249.0,GPS,8.0,STURE,2006-03-03 02:00:00,AIS
711500,2006-03-02 23:59:59,Class A,265542530,58.591353,16.198910,Unknown value,NaN,0.1,188.4,NaN,SMA TEST 3,2.0,2.0,GPS,0.1,NaN,NaT,AIS
711501,2006-03-02 23:59:59,Class A,258808000,55.257650,14.179800,Under way using engine,NaN,10.7,52.0,54.0,LISBETH,12.0,76.0,GPS,4.8,KALMAR,2006-03-01 12:00:00,AIS
711502,2006-03-02 23:59:59,Class A,265505410,59.290005,18.915278,Unknown value,NaN,0.1,193.8,NaN,LOTS 151,4.0,12.0,GPS,25.5,NaN,NaT,AIS


In [10]:
data1 = data1.sort_values(by=['MMSI','Timestamp'])

In [11]:
groups = tuple(data1.groupby('MMSI'))

In [12]:
len(groups)

2258

In [13]:
chunk_size = 10

In [14]:
chunk = [groups[i:i+chunk_size] for i in range(0, len(groups), chunk_size)]

In [15]:
chunk[3][0][0]

210374000

In [16]:
loc_anamoly_container = pd.DataFrame()
mmsi_ship=[]
total_records_ship = []
flag_count_ship =[]
mega_jump_ship = []
final_flag_ship = []
final_jump_count_ship = []

In [17]:
container = chunk[225]
mmsi , data2 = container[1]  
print(len(chunk), len(container))

  

226 8


Manually calculating chunk spoofing: 

In [18]:
for i in range(len(chunk)):
          container = chunk[i]
          for j in range(len(container)):
                
                mmsi , data2 = container[j]
                
        
        
            
            
                time2 = None
                lat1 = None
                lon1 = None
                count = 0
                flag_count = 0
                final_flag_count = 0
                jump_count = 0
                flag_sequence = 0
                flag_sequence_count = 0
                name = 0
                ob1 = None
                jump_sequence = None
                jump_sequence_count = None
                final_jump_count = 0
                
                for i2,row in data2.iterrows():
                    count+=1
                    
                    lat = row['Latitude']
                    lon = row['Longitude']
                    point = (lat,lon)   
                    time1 = row['Timestamp']
                    speed = row['SOG'] * 1.852
                
                    if count>=2:
                        time_dif = (time1 - time2).total_seconds()/60           #in minutes
                        dis = geodesic(point, point1).kilometers 
                        max_can_go = 1 * time_dif                               #as at 30 knot(.92 km/m) speed, ship can go 1km max in 1 minutes
                        #print('Distance: ', dis,'KM','  Time difference: ',time_dif, 'maximum distance can cover in this time: ', max_can_go, 'Actual speed: ', 
                             # speed, 
                             # 'with actual speed should go: ', speed/60 * time_dif, time1)
                
                
                        #detecting mid level anamolies 
                        if   (dis > max_can_go*3) and (dis<max_can_go * 50)  :
                            
                            
                            flag_count+=1
                            ob = data.index[count]-1
                            #print(ob)
                            
                            if (flag_sequence == ob) :
                                flag_sequence_count += 1
                                #print('scon',flag_sequence_count)
                                
                            if (flag_sequence != ob) :
                                 flag_sequence_count = 0
                
                                    
                            if flag_sequence_count >=2:
                                final_flag_count+=1
                                flag_sequence_count = 0
                           
                
                            #print(flag_sequence)
                            flag_sequence = data.index[count]
                         
                            #print('flag  ')
                
                            #detecting jump (serious)
                
                        if dis > max_can_go * 50 :
                            jump_count+=1
                            ob1 = data.index[count]-1
                            
                            if (jump_sequence == ob1) :
                                jump_sequence_count += 1         
        
                            if (jump_sequence != ob1) :
                                 jump_sequence_count = 0
        
                            if jump_sequence_count >=1:
                                final_jump_count+=1
                                jump_sequence_count = 0
        
                            
                            jump_sequence = data.index[count]
        
        
        
                            
                            
                            
                            #print('Mega Jump  ')
                                  
                   
                    time2 = time1
                    lat1 = lat
                    lon1 = lon
                    point1 = (lat1,lon1)
                    
                
                
                #print('Total flag: ', flag_count,'  Mega Jump: ',jump_count, count) 
                #print('final_flag: ', final_flag_count,'\n\n\n', i ,'\n',j)
                
                mmsi_ship.append(mmsi)
                total_records_ship.append(count)
                flag_count_ship.append(flag_count)
                mega_jump_ship.append(jump_count)
                final_flag_ship.append(final_flag_count)
                final_jump_count_ship.append(final_jump_count)

In [107]:
loc_anamoly_container['mmsi_ship']= mmsi_ship
loc_anamoly_container['total_records_ship']= total_records_ship
loc_anamoly_container['flag_count_ship']= flag_count_ship
loc_anamoly_container['mega_jump_ship']= mega_jump_ship
loc_anamoly_container['final_flag_ship']= final_flag_ship
loc_anamoly_container['final_jump_count_ship']= final_jump_count_ship


In [108]:
with pd.option_context('Display.max_rows', None):
 display(loc_anamoly_container[(loc_anamoly_container['final_flag_ship']>=1) | (loc_anamoly_container['final_jump_count_ship']>=1) ])

,mmsi_ship,total_records_ship,flag_count_ship,mega_jump_ship,final_flag_ship,final_jump_count_ship
0,0,591,0,30,0,11
2,1193046,476,7,102,0,36
1390,259759000,52,1,3,0,1
1681,265539620,3202,767,3,301,0
2257,999999999,263,1,28,0,9


In [109]:
end = time.time()

print('Total time: ' , start-end)

Total time:  -43.66337490081787


Now, Multicompute each chunk----> To PyCharm